# 4. Knn prediction of used car prices

## Context

I have written a from scratch algorithm for k nearest neighbors (knn) classification and tested it out on the Iris dataset. Here I want to extend the functionality of the knn code to enable predicting a continuous variable, in this early case price of a used car.

## Work outline

I will be updating module code to include new functions from the notebook #3 and adding extension functions to enable predicting a continuous variable. The [used car dataset](https://www.engineeringbigdata.com/wp-content/uploads/usedcars.csv) is from `www.engineeringbigdata.com`.

### Steps

1. import module from local filepath; load local data file
2. split the dataset into training and test populations
3. run predict_category_from_knn on test set to generate predictions
4. find score for algorithm


## Result

TBD

## Next steps

Next I will consider how to scale and adjust weights to improve the model predictiveness.


## 1. import module from local filepath; load local data file

Also cleanup the dataset to prepare for applying functions


In [2]:
import csv
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np

from knn_base import euclidean_distance
from knn_base import find_k_neighbors
from knn_base import predict_category_from_knn

In [3]:
# load raw dataset
with open('data/usedcars.csv', 'r') as f:
    lines = csv.reader(f)
    dataset = list(lines)

In [4]:
dataset[:5]

[['year', 'model', 'price', 'mileage', 'color', 'transmission'],
 ['2011', 'SEL', '21992', '7413', 'Yellow', 'AUTO'],
 ['2011', 'SEL', '20995', '10926', 'Gray', 'AUTO'],
 ['2011', 'SEL', '19995', '7351', 'Silver', 'AUTO'],
 ['2011', 'SEL', '17809', '11613', 'Gray', 'AUTO']]

In [5]:
print Counter([row[0] for row in dataset])
print Counter([row[1] for row in dataset])
print Counter([row[2] for row in dataset])
print Counter([row[3] for row in dataset])
print Counter([row[4] for row in dataset])
print Counter([row[5] for row in dataset])

Counter({'2010': 49, '2009': 42, '2011': 16, '2008': 14, '2007': 11, '2006': 6, '2000': 3, '2004': 3, '2005': 2, '2002': 1, '2003': 1, '2001': 1, 'year': 1, '2012': 1})
Counter({'SE': 78, 'SES': 49, 'SEL': 23, 'model': 1})
Counter({'12995': 7, '13992': 4, '13995': 4, '16995': 4, '14992': 3, '10995': 3, '13950': 3, '15999': 2, '15992': 2, '14999': 2, '6980': 2, '9999': 2, '9995': 2, '16950': 2, '7995': 2, '10955': 2, '15499': 2, '14495': 2, '12500': 2, '14677': 1, '12777': 1, '17000': 1, '11980': 1, '11984': 1, '15688': 1, '14989': 1, '5980': 1, '14477': 1, '10815': 1, '8494': 1, '15995': 1, '13845': 1, '14355': 1, '6995': 1, '5995': 1, '12704': 1, '15298': 1, '15500': 1, '13687': 1, '14990': 1, '14995': 1, '15988': 1, '20995': 1, '12780': 1, '15980': 1, '4899': 1, '9651': 1, '17500': 1, '13425': 1, '9992': 1, '12988': 1, '21992': 1, '13991': 1, '17495': 1, '13997': 1, '10979': 1, '13999': 1, '14699': 1, '13384': 1, '8800': 1, '13383': 1, '14299': 1, '12595': 1, '15889': 1, '12992': 1, 

In [6]:
# going to put numerical features first
# then categorical in increasing numbers of categories
# last row is price
new_dataset = []
for row in dataset:
    new_row = [row[3], row[0], row[5], row[1], row[4], row[2]]
    new_dataset.append(new_row)

In [7]:
raw_dataset_array = np.array(new_dataset)

In [8]:
raw_dataset_array[:5]

array([['mileage', 'year', 'transmission', 'model', 'color', 'price'],
       ['7413', '2011', 'AUTO', 'SEL', 'Yellow', '21992'],
       ['10926', '2011', 'AUTO', 'SEL', 'Gray', '20995'],
       ['7351', '2011', 'AUTO', 'SEL', 'Silver', '19995'],
       ['11613', '2011', 'AUTO', 'SEL', 'Gray', '17809']], dtype='|S12')

In [10]:
print Counter([row[1] for row in dataset])
print ''
print Counter([row[4] for row in dataset])
print ''
print Counter([row[5] for row in dataset])

Counter({'SE': 78, 'SES': 49, 'SEL': 23, 'model': 1})

Counter({'Black': 35, 'Silver': 32, 'Red': 25, 'Blue': 17, 'Gray': 16, 'White': 16, 'Green': 5, 'Yellow': 3, 'Gold': 1, 'color': 1})

Counter({'AUTO': 128, 'MANUAL': 22, 'transmission': 1})


In [11]:
# need to turn all of the categorical variables into (0, 1) items
transmission_lookup = {
    'AUTO': 0,
    'MANUAL': 1
}
model_lookup = {
    'SE': (1, 0, 0),
    'SES': (0, 1, 0),
    'SEL': (0, 0, 1)
}
color_lookup = {
    'Black': (1, 0, 0, 0, 0, 0, 0, 0, 0),
    'Silver': (0, 1, 0, 0, 0, 0, 0, 0, 0),
    'Red': (0, 0, 1, 0, 0, 0, 0, 0, 0),
    'Blue': (0, 0, 0, 1, 0, 0, 0, 0, 0),
    'Gray': (0, 0, 0, 0, 1, 0, 0, 0, 0),
    'White': (0, 0, 0, 0, 0, 1, 0, 0, 0),
    'Green': (0, 0, 0, 0, 0, 0, 1, 0, 0,),
    'Yellow': (0, 0, 0, 0, 0, 0, 0, 1, 0),
    'Gold': (0, 0, 0, 0, 0, 0, 0, 0, 1),
}

In [19]:
categorical_features = [['MANUAL', 'SE', 'SES', 'SEL',
                        'Black', 'Silver', 'Red', 'Blue', 'Gray',
                        'White', 'Green', 'Yellow', 'Gold']]
for row in raw_dataset_array[1:]:
    new_row = []
    new_row.append(transmission_lookup[row[2]])
    new_row.extend(model_lookup[row[3]])
    new_row.extend(color_lookup[row[4]])
    categorical_features.append(new_row)

In [21]:
categorical_features[:3]

[['MANUAL',
  'SE',
  'SES',
  'SEL',
  'Black',
  'Silver',
  'Red',
  'Blue',
  'Gray',
  'White',
  'Green',
  'Yellow',
  'Gold'],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [22]:
categorical_features_array = np.array(categorical_features)

In [23]:
header = list(raw_dataset_array[0])[:2] + categorical_features[0] + [raw_dataset_array[0][5]]

In [24]:
header

['mileage',
 'year',
 'MANUAL',
 'SE',
 'SES',
 'SEL',
 'Black',
 'Silver',
 'Red',
 'Blue',
 'Gray',
 'White',
 'Green',
 'Yellow',
 'Gold',
 'price']

In [27]:
raw_dataset_array[1:, :2].ndim

2

In [28]:
categorical_features_array[1:].ndim

2

In [30]:
raw_dataset_array[1:, 5].ndim

1

In [35]:
dataset_array1 = np.concatenate(
    (raw_dataset_array[1:, :2],
    categorical_features_array[1:],
    raw_dataset_array[1:, 5:]),
    axis=1
)

In [36]:
dataset_array1[:5]

array([['7413', '2011', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
        '0', '1', '0', '21992'],
       ['10926', '2011', '0', '0', '0', '1', '0', '0', '0', '0', '1',
        '0', '0', '0', '0', '20995'],
       ['7351', '2011', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0',
        '0', '0', '0', '19995'],
       ['11613', '2011', '0', '0', '0', '1', '0', '0', '0', '0', '1',
        '0', '0', '0', '0', '17809'],
       ['8367', '2012', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1',
        '0', '0', '0', '17500']], dtype='|S12')

In [37]:
dataset_array2 = np.array(dataset_array1, dtype=float)

In [38]:
dataset_array2[:5]

array([[7.4130e+03, 2.0110e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        2.1992e+04],
       [1.0926e+04, 2.0110e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        2.0995e+04],
       [7.3510e+03, 2.0110e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.9995e+04],
       [1.1613e+04, 2.0110e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.7809e+04],
       [8.3670e+03, 2.0120e+03, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 

In [39]:
dataset_array2.shape

(150, 16)

In [40]:
test_knn = find_k_neighbors(dataset_array2[0], dataset_array2[1:], 4, 15)

In [41]:
test_knn

[array([7.3510e+03, 2.0110e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.9995e+04]),
 array([7.7840e+03, 2.0100e+03, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.5499e+04]),
 array([8.367e+03, 2.012e+03, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 1.750e+04]),
 array([9.1990e+03, 2.0110e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.6992e+04])]

In [42]:
predicted_price_from_test = 0.25 * sum([row[15] for row in test_knn])

In [43]:
predicted_price_from_test

17496.5

In [44]:
actual_price = dataset_array2[0][15]

In [45]:
actual_price

21992.0

In [46]:
raw_error = actual_price - predicted_price_from_test

In [47]:
raw_error

4495.5